# 네이버 웹툰 홈 화면의 장르별 TOP10 데이터를 크롤링 테스트 

In [1]:
import requests

In [28]:
url  =  "https://comic.naver.com/api/home/component?type=GENRE_RANK&genre=ACTION"

head  = {'User-Agent':
'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36'}


In [29]:
r = requests.get(url, headers=head)

In [12]:
r.status_code

200

In [31]:
r.json()

{'genreType': 'ACTION',
 'titleList': [{'titleId': 783053,
   'titleName': '김부장',
   'thumbnailUrl': 'https://image-comic.pstatic.net/webtoon/783053/thumbnail/thumbnail_IMAG21_d7732f14-26da-4e35-8762-660cc87b53f1.jpg',
   'thumbnailBadgeList': [],
   'up': False,
   'rest': False,
   'rank': 1},
  {'titleId': 758037,
   'titleName': '참교육',
   'thumbnailUrl': 'https://image-comic.pstatic.net/webtoon/758037/thumbnail/thumbnail_IMAG21_15cb2611-34c0-4f02-a689-41d0b1016579.jpg',
   'thumbnailBadgeList': [],
   'up': False,
   'rest': False,
   'rank': 2},
  {'titleId': 808198,
   'titleName': '촉법소년',
   'thumbnailUrl': 'https://image-comic.pstatic.net/webtoon/808198/thumbnail/thumbnail_IMAG21_2aabb538-5b1a-4d74-81ed-b41bb8a155ff.jpg',
   'thumbnailBadgeList': [],
   'up': True,
   'rest': False,
   'rank': 3},
  {'titleId': 736277,
   'titleName': '싸움독학',
   'thumbnailUrl': 'https://image-comic.pstatic.net/webtoon/736277/thumbnail/thumbnail_IMAG21_bbbe3f76-021e-4dbc-830a-4358c1abec0c.jpg',


In [20]:
get_url  =  "https://comic.naver.com/api/home/component?type=GENRE_RANK&genre={}"

In [18]:
genres = ['PURE','FANTASY','Action','DAILY','THRILL','COMIC','HISTORICAL','DRAMA','SENSIBILITY','SPORTS']

In [27]:
r.json()['titleList'][0]

{'titleId': 783053,
 'titleName': '김부장',
 'thumbnailUrl': 'https://image-comic.pstatic.net/webtoon/783053/thumbnail/thumbnail_IMAG21_d7732f14-26da-4e35-8762-660cc87b53f1.jpg',
 'thumbnailBadgeList': [],
 'up': False,
 'rest': False,
 'rank': 1}

## 각 장르별 데이터 추출 csv

In [4]:
from tqdm import tqdm
import requests
import pandas as pd

get_url  =  "https://comic.naver.com/api/home/component?type=GENRE_RANK&genre={}"
genres = ['PURE','FANTASY','ACTION','DAILY','THRILL','COMIC','HISTORICAL','DRAMA','SENSIBILITY','SPORTS']
df = pd.DataFrame()

for i in tqdm(genres):
    print(i)
    r =requests.get(get_url.format(i))
    data_list = []
    k = r.json()['titleList']

    for j in range(len(k)):
        temp = {
            'genre' : i,
            'webtoon_id' : k[j]['titleId'],
            'title' : k[j]['titleName'],
            'thumbnail_url' :k[j]['thumbnailUrl'],
            'rank'       : k[j]['rank']
        }
        data_list.append(temp)
    df = pd.concat([df, pd.DataFrame(data_list)])

df.to_csv("./webtoon_rank.csv", encoding="utf-8-sig", index=False)

 20%|██        | 2/10 [00:00<00:00, 19.59it/s]

PURE
FANTASY
ACTION
DAILY
THRILL


100%|██████████| 10/10 [00:00<00:00, 24.53it/s]

COMIC
HISTORICAL
DRAMA
SENSIBILITY
SPORTS


## DB에 저장

In [2]:
import pymysql
import csv

In [7]:
try:
    # 각 MySQL 환경에 맞게 host, user, password, db 값 입력.
    con = pymysql.connect(host='localhost', user='root', password='',  db='', charset='utf8', port=3306)
    cur = con.cursor()
except Exception as e:
    print (e)


In [8]:
# Webtoon_rank 테이블 있는 지 확인
check = cur.execute("show tables like 'Webtoon_rank'")

# 테이블이 없다면 테이블 생성
if check == False:
    cr_table = """CREATE TABLE webtoon_rank(
                    numb BIGINT PRIMARY KEY NOT NULL AUTO_INCREMENT,
                    genre VARCHAR(255),
                    webtoon_id INT(11) NOT NULL,
                    title VARCHAR(255),
                    thumbnail_url VARCHAR(255),
                    webtoon_rank VARCHAR(255)
                    )
                """
    cur.execute(cr_table)

In [9]:
sql = "INSERT INTO webtoon_rank(genre, webtoon_id, title, thumbnail_url, webtoon_rank) VALUES(%s, %s, %s, %s, %s)"

with open("./webtoon_rank.csv", "r", encoding='utf-8-sig') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    for idx,row in enumerate(reader):
        print(row)
        if idx > 0: # csv 파일 내 컬럼 값들을 제외시키기 위해.
            try:
                cur.execute(sql, row)
            except Exception as e:
                print(e)
con.commit()

['genre', 'webtoon_id', 'title', 'thumbnail_url', 'rank']
['PURE', '802293', '앞집나리', 'https://image-comic.pstatic.net/webtoon/802293/thumbnail/thumbnail_IMAG21_a3e4736d-dfd6-4e76-aec6-b145f4123845.jpg', '1']
['PURE', '807859', '성스러운 작가생활', 'https://image-comic.pstatic.net/webtoon/807859/thumbnail/thumbnail_IMAG21_f76458b0-e807-480e-b44c-3437d9888a75.jpg', '2']
['PURE', '735661', '재혼 황후', 'https://image-comic.pstatic.net/webtoon/735661/thumbnail/thumbnail_IMAG21_4b3c44a0-f286-4878-bac3-84c3ec9dc0a1.jpg', '3']
['PURE', '797410', '남편을 죽여줘요', 'https://image-comic.pstatic.net/webtoon/797410/thumbnail/thumbnail_IMAG21_9852442e-217e-4ec7-92a7-21c0ada85dc7.jpg', '4']
['PURE', '790245', '어쩌다보니 천생연분', 'https://image-comic.pstatic.net/webtoon/790245/thumbnail/thumbnail_IMAG21_3545003845982959922.jpg', '5']
['PURE', '796867', '버림받은 왕녀의 은밀한 침실', 'https://image-comic.pstatic.net/webtoon/796867/thumbnail/thumbnail_IMAG21_77aa5064-b42b-4838-912e-2c6266c53a74.jpg', '6']
['PURE', '813552', '우아한 욕망', 'ht

In [15]:
print(row)

SPORTS,775140,10



In [36]:
with open("./webtoon.csv", "r", encoding='utf-8-sig') as csvfile:
    # reader = csv.reader(csvfile, delimiter=',')
    for idx,row in enumerate(csvfile):
        print(row)
        print(type(row))

WebtoonId,Title,Writer,Painter,Age,Favorite,StarScore,ThumbnailUrl,PublishDay,HashTags

<class 'str'>
769209,화산귀환,LICO,LICO,RATE_15,1137505,9.93,https://image-comic.pstatic.net/webtoon/769209/thumbnail/thumbnail_IMAG21_3511dcdd-6e33-4171-8839-598d6d266215.jpg,WEDNESDAY,"무협/사극,세계관,동양풍판타지,회귀,힘숨찐,이세계,먼치킨,소설원작"

<class 'str'>
811721,무직백수 계백순,지발,지발,RATE_15,130083,9.79,https://image-comic.pstatic.net/webtoon/811721/thumbnail/thumbnail_IMAG21_9a2a959a-666b-4156-8e4f-db64dfe319c6.jpg,"WEDNESDAY,SUNDAY","개그,요즘핫한추천작"

<class 'str'>
747269,전지적 독자 시점,UMI,슬리피-C,RATE_15,1014896,9.94,https://image-comic.pstatic.net/webtoon/747269/thumbnail/thumbnail_IMAG21_aabd9952-ff45-47a2-a543-33f19a5c6708.jpg,WEDNESDAY,"판타지,세계관,레드아이스스튜디오,아포칼립스,게임판타지,헌터물,이능력배틀물,먼치킨,구원서사,소설원작"

<class 'str'>
807859,성스러운 작가생활,이작가야,이작가야,RATE_18,175479,9.95,https://image-comic.pstatic.net/webtoon/807859/thumbnail/thumbnail_IMAG21_f76458b0-e807-480e-b44c-3437d9888a75.jpg,WEDNESDAY,"로맨스,요즘핫한추천작,로맨스코미디,직업드라마,전남친,까칠남,성인웹툰"

<class 'str'>
